In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping,CSVLogger
from keras.layers.advanced_activations import LeakyReLU
from datetime import datetime

#This is the basic model using unresized 200x300 images. These settings produced the model with the best abnormal call accuracy

#Variables to edit-----------------
epochs=500                #Number of epochs for training
filenameprefix = 'CNN200x300ep'+str(epochs) #Start of filename that will be used in last saved model after training

learningrate = 0.0005  #Learning rate for optimizer if used for adam optimizer

espatience = 30          #tensorboard Earlystop function patience value --Use if EarlyStopping is used
#End of variables to edit----------
filename = filenameprefix + str(datetime.now().strftime('%Y%m%d-%H%M%S'))

#Tensorboard functions
tensorboard = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

checkpointvacc  = ModelCheckpoint('weights/{epoch:02d}.h5'
                             , monitor='val_acc', mode='max', save_best_only=True)
checkpointvloss  = ModelCheckpoint('weights/{epoch:02d}.h5'
                             , monitor='val_loss', mode='min', save_best_only=True)
checkpointloss  = ModelCheckpoint('weights/{epoch:02d}.h5'
                             , monitor='loss', mode='min', save_best_only=True)
earlystop   = EarlyStopping(monitor='val_acc', patience=espatience, mode='auto', verbose=1)
csv_logger = CSVLogger(filename+'training.csv', separator=',')


train_data = ImageDataGenerator(rescale=1./255)
test_data = ImageDataGenerator(rescale=1./255)

#End of tensorboard functions
train_generator = train_data.flow_from_directory('../images/train',batch_size=40,class_mode='binary',target_size=(200,300))
test_generator = test_data.flow_from_directory('../images/test',batch_size=32,class_mode='binary',target_size=(200,300))
label_map = (train_generator.class_indices)#Stores classes and their values. Can be displayed on screen if needed

#Model Layers-Layers represent the factors of 200 & 300 for easier calculations
model = Sequential()
model.add(Conv2D(96, (5, 5), strides=5, input_shape=(200, 300, 3)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(128, (5, 5), strides=5))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))      
model.add(Flatten())
model.add(Dense(768, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
#End of Model layers------------------------------------------------------------
#adam = optimizers.adam(lr=learningrate) #Adam was used in testing most of the time but sgd produced the best results at the end
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
history=model.fit_generator( train_generator,
        steps_per_epoch=60,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=20, callbacks=[checkpointvacc,checkpointvloss])#Add tensorboard functions here
        

#model.summary() #Displays the model structure
filename = filenameprefix + str(datetime.now().strftime('%Y%m%d-%H%M%S')) #Get time and date now to add to filename
model.save('weights/%s.h5' % filename) #Save model

#------------------------------------------------------------------------------
# Plot training & validation accuracy values by history saved-doesn't use tensorboard
plt.figure(1,figsize=(10,6))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(2,figsize=(10,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()



In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import os, shutil
import numpy as np
from skimage.transform import resize
#Testing code 
#This program reads models from a folder (example: Those saved by Modelcheckpoint in tensorboard) and runs predictions on a
#folder(s) of images. Output displays the # of results from the cutoff to 1 individually and the % of the results at that cutoff 
imageheight=200
imagewidth=300
cutoff=0.7 #Above this value is an echolcation + below is abnormal file-Used for displaying the min. cutoff to calculate from
mfolder = "./weights/" #Folder with saved models in h5 format
ifolder = "../abcopy/a/" #Folder with images to test model with
foldercheck2 = True #Do you want to check the 2nd folder of images?
ifolder2 = "../images/train/echolocation/" #Folder2 with images to test model with
foldercheck3 = False #Do you want to check the 3rd folder of images?
ifolder3 = "../abcopy/a/" #Folder3 with images to test model with
mincheck = 200 #Min starting model which represents the epoch that it was created-Models below this will be deleted from folder

modcount=0 #Counts the model number for display purposes
for mdl in range(mincheck+1): #Remove unwanted models less than this variable
    if os.path.exists(mfolder+str(mdl)+".h5"):
        os.remove(mfolder+str(mdl)+".h5")
    if os.path.exists(mfolder+"0"+str(mdl)+".h5"):
        os.remove(mfolder+"0"+str(mdl)+".h5")   
print('Reading directories from model and image folders')
mfilelist = os.listdir(mfolder) #Make list of model filenames
ifilelist = os.listdir(ifolder) #Make list of image filenames
if foldercheck2 == True:
    ifilelist2 = os.listdir(ifolder2) #Make list of image filenames for 2nd scan
if foldercheck3 == True:
    ifilelist3 = os.listdir(ifolder3) #Make list of image filenames for 3rd scan
for ipremove in mfilelist: #Remove .ipynb_checkpoints folder which may get saved in models folder
    if '{}'.format(ipremove[0:6]) == '.ipynb':
        shutil.rmtree(mfolder+ipremove)
mfilelist = os.listdir(mfolder) #Make list of model filenames again if .ipynb_checkpoints existed before 
print('Processing {0:2d} images in {1:2d} models-------------------------'.format(len(ifilelist),len(mfilelist)))
cutoffdiff = int((1-cutoff)*10) #Calculates the difference between the cutoff and 1-Used for calculating & displaying accuracies
for mdl in range(len(mfilelist)):
    modcount += 1
    model = load_model(mfolder+mfilelist[mdl])
    #model.summary() #Displays model layers and info
    abcount = [0] * cutoffdiff 
    for inum in range(len(ifilelist)):
        image = load_img(ifolder + ifilelist[inum], target_size=(imageheight,imagewidth))
        imageresize = image.resize((imagewidth, imageheight)) #Resize image for desired CNN model
        imagearray = img_to_array(imageresize) #Make image into an array
        imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
        imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
        prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
        for abdetermine in range(cutoffdiff):
            if prediction[0]<(cutoff+(abdetermine*0.1)):
                abcount[abdetermine]=abcount[abdetermine]+1
    print('{0:2d}. of {1:2d} Model: {2:s}'.format(modcount,len(mfilelist),mfilelist[mdl]))
    for abprint in range(cutoffdiff):
        print('Abnormal @ cutoff {0:2.1f}: {1:2d} at {2:4.2f}% Echolocation count: {3:2d} at {4:4.2f}%'.format(cutoff+(abprint*0.1),
        abcount[abprint],abcount[abprint]*100/len(ifilelist),(len(ifilelist)-abcount[abprint]),             
        (len(ifilelist)-abcount[abprint])*100/len(ifilelist)))
    if foldercheck2 == True:
        #print('Processing {0:2d} images in each model'.format(len(ifilelist2)))
        abcount = [0] * cutoffdiff
        for inum in range(len(ifilelist2)):
            image = load_img(ifolder2 + ifilelist2[inum], target_size=(imageheight,imagewidth))
            imageresize = image.resize((imagewidth, imageheight)) #Resize image for desired CNN model
            imagearray = img_to_array(imageresize) #Make image into an array
            imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
            imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
            prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
            for abdetermine in range(cutoffdiff):
                if prediction[0]<(cutoff+(abdetermine*0.1)):
                    abcount[abdetermine]=abcount[abdetermine]+1
        print('Folder 2 results:')
        for abprint in range(cutoffdiff):
            print('Abnormal @ cutoff {0:2.1f}: {1:2d} at {2:4.2f}% Echolocation count: {3:2d} at {4:4.2f}%'.format(cutoff+(abprint*0.1),
            abcount[abprint],abcount[abprint]*100/len(ifilelist2),(len(ifilelist2)-abcount[abprint]),             
            (len(ifilelist2)-abcount[abprint])*100/len(ifilelist2)))
    if foldercheck3 == True:
        #print('Processing {0:2d} images in each model'.format(len(ifilelist3)))
        abcount = [0] * cutoffdiff
        for inum in range(len(ifilelist3)):
            image = load_img(ifolder3 + ifilelist3[inum], target_size=(imageheight,imagewidth))
            imageresize = image.resize((imagewidth, imageheight)) #Resize image for desired CNN model
            imagearray = img_to_array(imageresize) #Make image into an array
            imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
            imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
            prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
            for abdetermine in range(cutoffdiff):
                if prediction[0]<(cutoff+(abdetermine*0.1)):
                    abcount[abdetermine]=abcount[abdetermine]+1
        print('Folder 3 results:')
        for abprint in range(cutoffdiff):
            print('Abnormal @ cutoff {0:2.1f}: {1:2d} at {2:4.2f}% Echolocation count: {3:2d} at {4:4.2f}%'.format(cutoff+(abprint*0.1),
            abcount[abprint],abcount[abprint]*100/len(ifilelist3),(len(ifilelist3)-abcount[abprint]),             
            (len(ifilelist3)-abcount[abprint])*100/len(ifilelist3)))
    print('End of model {0:s} predictions-----------------------'.format(mfilelist[mdl]))
print('Finished')